In [14]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [15]:
# Problem data
pre_order_cost = 95
scenarios = range(1, 17)
scenario_prob = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
scenario_demand = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]

# Local supplier data
local_price = [120, 105, 110]
local_min_order = [0, 70, 40]

# Create model
m = gp.Model("Coffee_Ordering")

# Decision variables
x = m.addVar(lb=0, vtype=GRB.INTEGER, name="Pre_order")
y = m.addVars(scenarios, 3, lb=0, vtype=GRB.INTEGER, name="Order")

# Rename variables to reflect the supplier and scenario
for i in scenarios:
    for j in range(3):
        y[i, j].VarName = f"Order_{i}_{'Phil_Sebastian' if j == 0 else 'Rosso' if j == 1 else 'Monogram'}"

rosso_bin = m.addVars(scenarios, vtype=GRB.BINARY, name="Rosso_Binary")
monogram_bin = m.addVars(scenarios, vtype=GRB.BINARY, name="Monogram_Binary")

# Objective function
m.setObjective(pre_order_cost * x + gp.quicksum(scenario_prob[i-1] * gp.quicksum(local_price[j] * y[i, j] for j in range(3)) for i in scenarios), GRB.MINIMIZE)

# Constraints
for i in scenarios:
    # Demand satisfaction constraint
    m.addConstr(x + gp.quicksum(y[i, j] for j in range(3)) >= scenario_demand[i-1], name=f"Demand_{i}")
    
    # Minimum order constraints for local suppliers
    m.addConstr(y[i, 1] >= local_min_order[1] * rosso_bin[i], name=f"Min_order_Rosso_{i}")
    m.addConstr(y[i, 2] >= local_min_order[2] * monogram_bin[i], name=f"Min_order_Monogram_{i}")

# Solve model
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print(f"Optimal solution found.")
    print(f"Pre-order amount: {x.x} gallons")
    print(f"Total cost: ${m.objVal:.2f}")
    for i in scenarios:
        print(f"Scenario {i}: Phil & Sebastian: {y[i,0].x} gallons, Rosso: {y[i,1].x} gallons, Monogram: {y[i,2].x} gallons")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 48 rows, 81 columns and 128 nonzeros
Model fingerprint: 0xc895d427
Variable types: 0 continuous, 81 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]
Found heuristic solution: objective 12611.500000
Presolve removed 33 rows and 65 columns
Presolve time: 0.00s
Presolved: 15 rows, 16 columns, 30 nonzeros
Found heuristic solution: objective 11135.500000
Variable types: 0 continuous, 16 integer (0 binary)

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Bes

In [16]:
# Function to solve model for a single scenario demand
def solve_scenario(demand):
    model = gp.Model()
    x = model.addVar(lb=0, vtype=GRB.INTEGER, name="Pre_order")
    y = model.addVars(3, lb=0, vtype=GRB.INTEGER, name="Supplier_Orders")
    rosso_bin = model.addVar(vtype=GRB.BINARY, name="Rosso_Binary")
    monogram_bin = model.addVar(vtype=GRB.BINARY, name="Monogram_Binary")

    model.setObjective(pre_order_cost * x + gp.quicksum(local_price[j] * y[j] for j in range(3)), GRB.MINIMIZE)
    model.addConstr(x + gp.quicksum(y[j] for j in range(3)) >= demand, name="Demand")
    model.addConstr(y[1] >= local_min_order[1] * rosso_bin, name="Min_order_Rosso")
    model.addConstr(y[2] >= local_min_order[2] * monogram_bin, name="Min_order_Monogram")

    model.optimize()
    return model.objVal if model.status == GRB.OPTIMAL else float('inf')

# Compute EVPI
scenario_costs = [solve_scenario(demand) for demand in scenario_demand]
evpi = sum(prob * cost for prob, cost in zip(scenario_prob, scenario_costs)) - m.objVal


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 6 columns and 8 nonzeros
Model fingerprint: 0x64779207
Variable types: 0 continuous, 6 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 9e+01]
Found heuristic solution: objective 9900.0000000
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 8550 9900 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.550000000000e+03, best bound 8.550000000000e+03, gap 0.0000%
Gurobi Optimizer version 11.0.0 bui

In [19]:
def solve_scenario(demand, pre_order_cost, local_price, local_min_order):
    # Create model
    m = gp.Model(f"Scenario_{demand}")

    # Decision variables
    x = m.addVar(lb=0, vtype=GRB.INTEGER, name="Pre_order")
    y = m.addVars(3, lb=0, vtype=GRB.INTEGER, name=["Phil_Sebastian", "Rosso", "Monogram"])
    rosso_bin = m.addVar(vtype=GRB.BINARY, name="Rosso_Binary")
    monogram_bin = m.addVar(vtype=GRB.BINARY, name="Monogram_Binary")

    # Objective function
    m.setObjective(pre_order_cost * x + gp.quicksum(local_price[j] * y[j] for j in range(3)), GRB.MINIMIZE)

    # Constraints
    m.addConstr(x + gp.quicksum(y[j] for j in range(3)) >= demand, name="Demand")
    m.addConstr(y[1] >= local_min_order[1] * rosso_bin, name="Min_order_Rosso")
    m.addConstr(y[2] >= local_min_order[2] * monogram_bin, name="Min_order_Monogram")

    # Solve model
    m.optimize()

    return m.objVal

In [20]:
# Compute WS and EVPI
ws_objvals = [solve_scenario(demand, pre_order_cost, local_price, local_min_order) for demand in scenario_demand]
ws = sum(prob * obj for prob, obj in zip(scenario_prob, ws_objvals))
evpi = ws - m.objVal

print(f"WS: ${ws:.2f}")
print(f"EVPI: ${evpi:.2f}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 6 columns and 8 nonzeros
Model fingerprint: 0x64779207
Variable types: 0 continuous, 6 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 9e+01]
Found heuristic solution: objective 9900.0000000
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 8550 9900 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.550000000000e+03, best bound 8.550000000000e+03, gap 0.0000%
Gurobi Optimizer version 11.0.0 bui

In [17]:
# Deterministic model based on mean demand
mean_demand = sum(prob * demand for prob, demand in zip(scenario_prob, scenario_demand))
det_model = gp.Model("Deterministic_Model")
x_det = det_model.addVar(lb=0, vtype=GRB.INTEGER, name="Pre_order")

det_model.setObjective(pre_order_cost * x_det, GRB.MINIMIZE)
det_model.addConstr(x_det >= mean_demand, name="Mean_Demand")
det_model.optimize()

# Compute VSS
vss = m.objVal - (det_model.objVal if det_model.status == GRB.OPTIMAL else float('inf'))


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0xea7cade3
Variable types: 0 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 10925.000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 10925 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.092500000000e+04, best bound 1.092500000000e+04, gap 0.0000%


In [18]:
if m.status == GRB.OPTIMAL and det_model.status == GRB.OPTIMAL:
    print(f"Optimal Stochastic Solution cost: ${m.objVal:.2f}")
    print(f"Deterministic Model cost (using mean demand): ${det_model.objVal:.2f}")
    print(f"EVPI: ${evpi:.2f}")
    print(f"VSS: ${vss:.2f}")
else:
    print("One of the models did not achieve an optimal solution.")


Optimal Stochastic Solution cost: $11135.50
Deterministic Model cost (using mean demand): $10925.00
EVPI: $-243.75
VSS: $210.50


In [26]:
print("Optimal pre-order amount: {} gallons".format(x.X))
print("Total expected cost: ${:.2f}".format(m.ObjVal))


Optimal pre-order amount: 95.0 gallons
Total expected cost: $11135.50


In [28]:
def find_upper_limit(low, high, tolerance=0.01):
    while high - low > tolerance:
        mid = (low + high) / 2
        m.setObjective(mid * x + gp.quicksum(scenario_prob[i-1] * gp.quicksum(local_price[j] * y[i, j] for j in range(3)) for i in scenarios), GRB.MINIMIZE)
        m.optimize()
        if x.x > 0:
            low = mid
        else:
            high = mid
    return low

upper_limit = find_upper_limit(95, 200)
print(f"The upper limit for the cost of pre-ordering coffee is ${upper_limit:.2f} per gallon.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 48 rows, 81 columns and 128 nonzeros
Model fingerprint: 0x10f561ea
Variable types: 0 continuous, 81 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]

Loaded MIP start from previous solve with objective 16123

Presolve removed 33 rows and 65 columns
Presolve time: 0.00s
Presolved: 15 rows, 16 columns, 30 nonzeros
Variable types: 0 continuous, 16 integer (0 binary)
Found heuristic solution: objective 16071.050000

Root relaxation: objective 1.203825e+04, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

In [30]:
print("The upper limit for the cost of pre-ordering coffee is $117.19 per gallon.")

The upper limit for the cost of pre-ordering coffee is $117.19 per gallon.
